In [12]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output, Video
import os

In [2]:
au = 1.496e11
G = 6.67e-11

In [3]:
class particle:
    def __init__(self, pos, velocity=0., mass=1.):
        self.pos = float(pos)
        self.velocity = float(velocity)
        self.mass = float(mass)
    
    def update_grav(self, accel):
        self.velocity += accel
        
    def move(self):
        self.pos += self.velocity

In [4]:
def get_colours(palette, n):
    clrs = []
    cmap = matplotlib.cm.get_cmap(palette, n)
    for i in range(cmap.N):
        rgb = cmap(i)[:3]
        clrs.append(matplotlib.colors.rgb2hex(rgb))
    return clrs

In [5]:

def grav(part_a, part_b):
    dist = part_b.pos - part_a.pos
    if dist != 0:
        on_a = G * part_b.mass / dist
    else:
        on_a = 0
    part_a.update_grav(on_a)
    part_b.update_grav(-on_a)

In [6]:
def step(parts):
    for aidx, aval in enumerate(parts):
        for bidx, bval in enumerate(parts[aidx:]):
            grav(aval, bval)
    for i in parts:
        i.move()

In [57]:
def plot_parts(parts, save=None):
    fig = plt.figure(figsize=(10, 10))
    plt.xlim(-5*au, 5*au)
    plt.ylim(-au, au)
    plt.scatter(*zip(*[(i.pos, i.velocity) for i in parts]), c=clrs)
    if save:
        plt.savefig(save)
    plt.close()

In [53]:
def integrate(parts, steps, savedir=None, start=0):
    if start == 0:
        !rm {savedir}/fig_*.png
    for i in np.arange(start, start+steps):
        if savedir:
            pth = f'{savedir}/fig_{i+1:04}.png'
        plot_parts(parts, save=pth)
        step(parts)
    !ffmpeg -y -framerate 30 -pattern_type glob -i '{savedir}/fig_*.png' -c:v libx264 -pix_fmt yuv420p {savedir}/out.mp4

In [66]:
init_parts = []
n_bodies = 10
workingdir = 'grav'

init_pos = np.linspace(-(n_bodies // 2)*au, (n_bodies // 2)*au, n_bodies)
clrs = get_colours('inferno', n_bodies)

for pos, clr in zip(init_pos, clrs):
    part = particle(pos, mass=1e30)
    init_parts.append(part)

In [67]:
integrate(init_parts, 250, workingdir, start=0)

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.16)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.2 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.0.1.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.0.1.jdk/Contents/Home/include/darwin -fno-stack-check' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enab

In [68]:
Video(f'{workingdir}/out.mp4')